<a href="https://colab.research.google.com/github/Latiefabdul/Machine_Learning-With_TensorFlow/blob/main/Final_Exam_5_Crypto_Prediction_Timeseries_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

No. 5 Time Series

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # Deep learning library
import matplotlib.pyplot as plt # Visualization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Cryptocurrency2020.csv')
df

,Unix Timestamp,Date,Symbol,Open,High,Low,Close,Volume
0,1.618880e+12,4/20/2021 0:02,BTCUSD,55717.47,55723.00,55541.69,55541.69,2.406734
1,1.618880e+12,4/20/2021 0:01,BTCUSD,55768.94,55849.82,55711.74,55717.47,0.573458
2,1.618880e+12,4/20/2021 0:00,BTCUSD,55691.79,55793.15,55691.79,55768.94,3.309904
3,1.618880e+12,4/19/2021 23:59,BTCUSD,55777.86,55777.86,55677.92,55691.79,0.684934
4,1.618880e+12,4/19/2021 23:58,BTCUSD,55803.50,55823.88,55773.08,55777.86,0.477225
...,...,...,...,...,...,...,...,...
658060,1.577840e+12,1/1/2020 0:04,BTCUSD,7163.60,7163.60,7163.00,7163.00,1.751844
658061,1.577840e+12,1/1/2020 0:03,BTCUSD,7163.00,7163.60,7163.00,7163.60,0.002706
658062,1.577840e+12,1/1/2020 0:02,BTCUSD,7163.30,7164.22,7163.00,7163.00,0.063904
658063,1.577840e+12,1/1/2020 0:01,BTCUSD,7163.30,7164.23,7163.30,7163.30,0.002646


In [ ]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle_buffer > 0:
        ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[-1:]))
    return ds.batch(batch_size).prefetch(1)

train, val = train_test_split(df.iloc[:, 3:], test_size=0.2, shuffle=False)
scaler = MinMaxScaler().fit(train)
train = scaler.transform(train)[:, 0]
val = scaler.transform(val)[:, 0]
train = windowed_dataset(train, window_size=64, batch_size=128, shuffle_buffer=1000)
val = windowed_dataset(val, window_size=64, batch_size=128, shuffle_buffer=1000)

In [ ]:
#Buatlah model dengan input layer disarankan menggunakan LSTM LSTM dan output layer 1 
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(64,1)),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32, activation=tf.keras.activations.swish),
    tf.keras.layers.Dense(1),
])

In [ ]:
#gunakan loss MAE, optimizer boleh bebas memilih
model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=tf.keras.optimizers.Adam(1e-3),
    metrics=["mae"]
)

In [ ]:
callbacks=[
        tf.keras.callbacks.EarlyStopping(
            patience=10, 
            restore_best_weights=True
        ), 
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            patience=5
        )
    ]

In [ ]:
#silakan menentukan banyaknya epoch sendiri
history = model.fit(
    train,
    epochs=50,
    validation_data=val,
    callbacks=[callbacks]
        )

Epoch 1/50
4113/4113 [==============================] - 588s 140ms/step - loss: 3.9183e-04 - mae: 0.0043 - val_loss: 3.7400e-06 - val_mae: 0.0025 - lr: 0.0010
Epoch 2/50
4113/4113 [==============================] - 573s 139ms/step - loss: 1.4752e-05 - mae: 0.0017 - val_loss: 3.4333e-07 - val_mae: 6.4629e-04 - lr: 0.0010
Epoch 3/50
4113/4113 [==============================] - 572s 139ms/step - loss: 3.3276e-05 - mae: 0.0023 - val_loss: 7.4143e-07 - val_mae: 0.0011 - lr: 0.0010
Epoch 4/50
4113/4113 [==============================] - 569s 138ms/step - loss: 7.8050e-06 - mae: 0.0014 - val_loss: 1.7348e-06 - val_mae: 0.0017 - lr: 0.0010
Epoch 5/50
4113/4113 [==============================] - 567s 138ms/step - loss: 1.4388e-05 - mae: 0.0015 - val_loss: 1.1681e-06 - val_mae: 0.0014 - lr: 0.0010
Epoch 6/50
4113/4113 [==============================] - 564s 137ms/step - loss: 7.8410e-06 - mae: 0.0013 - val_loss: 1.0091e-06 - val_mae: 0.0013 - lr: 0.0010
Epoch 7/50
4113/4113 [====================

In [ ]:
#save model menjadi .h5
model.save('model5.h5')